In [2]:
import pandas as pd
import numpy as np
import random as rd
import sklearn as sk

wineDF = pd.read_csv('winequality/winequality-red.csv',sep=';')

class ProcessingData:
    @staticmethod
    def ShuffleData(df: pd.DataFrame):
        for i in range(len(df)):
            rand_i=rd.randint(0,len(df)-1)
            df.loc[i],df.loc[rand_i] = df.loc[rand_i],df.loc[i]
        return df
    @staticmethod
    def SplitData(df:pd.DataFrame,x:int,y:int):
        if x+y>100:
            print("Podano zbyt dużą ilość danych do podziału")
            return
        train_set,test_set = pd.DataFrame(),pd.DataFrame()
        for i in range(len(df)):
            if rd.random()<x/100:
                train_set = train_set.append(df.loc[i])
            else:
                test_set = test_set.append(df.loc[i])
        return train_set,test_set
    @staticmethod
    def NormalizeData(df:pd.DataFrame):
        for atributte in df.columns[:-1]:
            mean = df[atributte].mean()
            stddev = df[atributte].std()
            df[atributte] = (df[atributte]-mean)/stddev
        return df


In [3]:
class softClassificator:
    @staticmethod
    def fit(df: pd.DataFrame):
        means = {}
        mins = {}
        maxes = {}
        for Class in df.iloc[:,-1].unique():
            means[Class] = {}
            mins[Class] = {}
            maxes[Class] = {}
            for atributte in df.columns:
                if atributte != df.columns[-1]:
                    means[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].mean()
                    mins[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].min()
                    maxes[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].max()
                    for row in df.index:
                        if df.loc[row,atributte] < means[Class][atributte]:
                            df.loc[row,atributte] = 0
                        else:
                            df.loc[row,atributte] = 1
        return df,means,mins,maxes

    @staticmethod
    def predict_sample(sample:pd.Series, means:dict, mins:dict, maxes:dict) -> str:
        for Class in means.keys():
            for atributte in means[Class].keys():
                if sample[atributte] < means[Class][atributte]:
                    sample[atributte] = 0
                else:
                    sample[atributte] = 1
        max_probability = 0
        max_Class = ""
        for Class in means.keys():
            probability = 1
            for atributte in means[Class].keys():
                if sample[atributte] == 0:
                    probability *= (1-means[Class][atributte])
                else:
                    probability *= means[Class][atributte]
            if probability > max_probability:
                max_probability = probability
                max_Class = Class
        return max_Class

    @staticmethod
    def accuracy(df: pd.DataFrame, means:dict, mins:dict, maxes:dict) -> float:
        correct = 0
        for row in df.index:
            if df.loc[row,df.columns[-1]] == softClassificator.predict_sample(df.loc[row],means,mins,maxes):
                correct += 1
        return f"{round(correct/len(df) * 100,2)}%"


In [24]:
wineDF = ProcessingData.NormalizeData(wineDF)
wineDF = ProcessingData.ShuffleData(wineDF)
train_set,test_set = ProcessingData.SplitData(wineDF,70,30)

In [27]:
for i in range(4):
    wineDF = ProcessingData.ShuffleData(wineDF)
    train_set,test_set = ProcessingData.SplitData(wineDF,70,30)
    train_set,means,mins,maxes = softClassificator.fit(train_set)
    print(softClassificator.accuracy(test_set,means,mins,maxes))

16.29%
32.48%
36.5%
30.51%
25.97%
